# Forecasting

In an operational setting we do hotstart for the nect forecast from the latest simulation.

- Requires:

    - The data folder. See [README](README.md) in this folder.

In [ ]:
#to use the full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Not neccessarily needed. Just to check version. Must be >= 0.6
import pyposeidon
pyposeidon.__version__

In [ ]:
import pyposeidon.model as pm

### Graphics

In [ ]:
import pyposeidon.utils.pplot # initialize matplolib accessor

In [ ]:
#optional
#pyposeidon.utils.pplot.__init__(dark_background=True) # set plt style for pplot graphics below

In [ ]:
import pyposeidon.utils.hplot # initialize pyviz accessor

- In order to have within the Notebook the executable binaries installed in the conda env of the Kernel we add the corresponding path to the env PATH. This is not needed when we are running from python.

In [ ]:
#Create a folder to save the output
import os
if not os.path.exists('test'):
            os.makedirs('test')

In [ ]:
import numpy
cpath = numpy.__path__[0].split('/lib/')[0] # get the current kernel path

os.environ['PATH'] += os.pathsep + cpath + '/bin' # add to PATH

In [ ]:
# use cartopy to get coastlines
import cartopy.feature as cf

cr='i'

coast = cf.NaturalEarthFeature(
    category='physical',
    name='land',
    scale='{}m'.format({'l':110, 'i':50, 'h':10}[cr]))

In [ ]:
import geopandas as gp
ne_i = gp.GeoDataFrame(geometry = [x for x in coast.geometries()])

## Setup

The minimum information required is an initial model and corresponding run. We set this one up. For more info see the [MODEL](./MODEL.ipynb) Notebook.


In [ ]:
#define in a dictionary the properties of the model..
model={'solver':'schism',
     'geometry':{'lon_min' : -25,'lon_max' : -12.,'lat_min' : 56.,'lat_max' : 74.}, 
#     'manning':.12, #uniform manning value
#     'windrot':0.00001, #uniform windrot value
     'coastlines':ne_i, # coastlines
     'mesh_generator' : 'jigsaw', # set grid generator 
     'start_date':'2018-10-1 0:0:0',
     'time_frame':'72H',
     'meteo_source' : './data/uvp_2018100100.grib', #path to meteo files
     'dem_source' : './data/dem.nc', #path to dem file
     'rpath':'./test/forecast/20181001.00/', #location of calc folder
#     'update':['all'], #set which component should be updated  (meteo,dem,model)
     'monitor':True, # get time series for observation points
     'parameters':{'dt':400, 'rnday':3., 'nhot':1, 'ihot':0,'nspool':9, 'ihfskip':36, 'nhot_write':108 } # set param.in components
    }

* Note that the parameters arguments will override relevant settings

## Initialize

In [ ]:
#initialize a model
b = pm.set(**model)

In [ ]:
#Execute
b.execute()

## Setup a (re)forecast

Let's assume we have a forecasting cycle where new meteo forcings become available every 12H.

We setup a forecast process as

In [ ]:
import pandas as pd

In [ ]:
# creating a time sequence of the runs
start_date = pd.to_datetime("2018-10-1 0:0:0")
end_date = pd.to_datetime("2018-10-2 12:0:0")
date_list = pd.date_range(start_date, end_date, freq="12H")
date_list

In [ ]:
# creating a sequence of folder to store the runs. In this case we name them after the date attribute.
# NOTE that the first folder is the fisrt run already perfomed!!
rpaths = ['./test/forecast/' + x.strftime("%Y%m%d.%H") + "/" for x in date_list]
rpaths

In [ ]:
# creating a sequence of folder from which we read the meteo.
mfiles = ['./data/' + x.strftime('uvp_%Y%m%d%H.grib') for x in date_list]
mfiles

So now we can execute the forecast sequencially as

In [ ]:
from pyposeidon.utils import cast

In [ ]:
for l in range(len(rpaths) - 1):

    fc = cast.set(solver='schism',
            model=b,
            ppath=rpaths[l],
            cpath=rpaths[l + 1],
            meteo=mfiles[l + 1],
            sdate=date_list[l + 1],
                ) 
    fc.run(execute=True)